一些phn替换

In [1]:
import json

def process_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if isinstance(data, list):
        for entry in data:
            if 'ipa_dysfluent' in entry:
                # 只处理音素替换
                entry['ipa_dysfluent'] = replace_phonemes(entry['ipa_dysfluent'])
    else:
        print("Error: JSON 数据不是列表类型。")
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

def replace_phonemes(ipa):
    # 替换 ʧ 为 tʃ 和 ʤ 为 dʒ
    ipa = ipa.replace('ʧ', 'tʃ')
    ipa = ipa.replace('ʤ', 'dʒ')
    ipa = ipa.replace('g', 'ɡ')
    ipa = ipa.replace('ɝ', 'ɜ')
    return ipa

file_path = '/home/jinming/claude/data/phn_ins.json'
process_json_file(file_path)

生成模型用的ipa，可以比较一下

In [ ]:
""" from https://github.com/keithito/tacotron """
from text import cleaners
from text.symbols import symbols


# Mappings from symbol to numeric ID and vice versa:
_symbol_to_id = {s: i for i, s in enumerate(symbols)}
_id_to_symbol = {i: s for i, s in enumerate(symbols)}


def text_to_sequence(text, cleaner_names):
  '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.
    Args:
      text: string to convert to a sequence
      cleaner_names: names of the cleaner functions to run the text through
    Returns:
      List of integers corresponding to the symbols in the text
  '''
  sequence = []

  clean_text = _clean_text(text, cleaner_names)
  for symbol in clean_text:
    symbol_id = _symbol_to_id[symbol]
    sequence += [symbol_id]
  return sequence

def text_to_sequence_phn(clean_text, cleaner_names):
  '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.
    Args:
      text: string to convert to a sequence
      cleaner_names: names of the cleaner functions to run the text through
    Returns:
      List of integers corresponding to the symbols in the text
  '''
  sequence = []

  for symbol in clean_text:
    symbol_id = _symbol_to_id[symbol]
    sequence += [symbol_id]
  return sequence


def cleaned_text_to_sequence(cleaned_text):
  '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.
    Args:
      text: string to convert to a sequence
    Returns:
      List of integers corresponding to the symbols in the text
  '''
  sequence = [_symbol_to_id[symbol] for symbol in cleaned_text]
  return sequence


def sequence_to_text(sequence):
  '''Converts a sequence of IDs back to a string'''
  result = ''
  for symbol_id in sequence:
    s = _id_to_symbol[symbol_id]
    result += s
  return result


def _clean_text(text, cleaner_names):
  for name in cleaner_names:
    cleaner = getattr(cleaners, name)
    if not cleaner:
      raise Exception('Unknown cleaner: %s' % name)
    text = cleaner(text)
  return text


In [ ]:

clean_text = _clean_text("kitten", hps.data.text_cleaners)
print(clean_text)


In [ ]:
import json
from multiprocessing import Pool

def process_entry(entry):
    if 'reference_text' in entry and 'ipa_reference' not in entry:
        # 生成 ipa_reference
        ipa_revised = _clean_text(entry['reference_text'], hps.data.text_cleaners)
        
        # 重新构建字典，确保键的顺序
        new_entry = {}
        for key in entry:
            new_entry[key] = entry[key]
            if key == 'dysfluent':
                new_entry['ipa_reference'] = ipa_revised
        # 如果 'ipa_dysfluent' 在原始字典中，继续添加后续键
        if 'ipa_dysfluent' in entry:
            new_entry['ipa_dysfluent'] = entry['ipa_dysfluent']
        # 更新 entry
        entry.clear()
        entry.update(new_entry)
    return entry

def process_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if isinstance(data, list):
        # 限制使用的 CPU 核心数量
        num_processes = 32
        with Pool(processes=num_processes) as pool:
            data = pool.map(process_entry, data)
    else:
        print("错误：JSON 数据不是列表类型。")
        return

    # 写回处理后的结果
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
# 文件路径
file_path = '/home/jinming/claude/data/phn_sub.json'

# 调用函数处理 JSON 文件
process_json_file(file_path)

# 真正的vits

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:

import matplotlib.pyplot as plt
import IPython.display as ipd
import csv
import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence, text_to_sequence_phn

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)  # text_to_sequence_phn
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
import json

with open('/home/jinming/claude/data/phn_del.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def move_field(item):
    if 'ipa_reference' in item and 'dysfluent' in item:
        ipa_value = item.pop('ipa_reference')
        keys = list(item.keys())
        index = keys.index('dysfluent') + 1
        new_item = {}
        for i, key in enumerate(keys):
            new_item[key] = item[key]
            if i == index - 1:
                new_item['ipa_reference'] = ipa_value
        return new_item
    return item

if isinstance(data, list):
    data = [move_field(item) for item in data]
elif isinstance(data, dict):
    data = move_field(data)

with open('/home/jinming/claude/data/phn_del.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

播放示例

In [ ]:
import os
import json
import IPython.display as ipd

# 路径配置
PATH_CONFIG = {
    'json_path': '/data/jinming/LLM/phn/phn_del_1.json',
    'save_dir': '/data/jinming/LLM/phn/phn_del',
}

# 加载数据
with open(PATH_CONFIG['json_path'], 'r', encoding='utf-8') as f:
    data = json.load(f)

# 播放指定 id 的音频并打印对应的 JSON 数据
def play_wav_by_id(id, save_dir, data):
    wav_path = os.path.join(save_dir, f"{id}.wav")
    
    # 查找对应的 JSON 数据
    entry = next((entry for entry in data if entry['id'] == id), None)
    
    if os.path.exists(wav_path):
        # 打印对应的 JSON 数据
        if entry:
            print(f"ID: {entry['id']}")
            print(f"Correct: {entry['correct']}")
            print(f"Dysfluent: {entry['dysfluent']}")
            print(f"Label: {entry['label']}")
            print(f"reference_text: {entry['reference_text']}")
        else:
            print(f"No data found for ID {id} in the JSON file.")
        
        # 使用 IPython.display.Audio 播放音频
        return ipd.Audio(wav_path, rate=22050, autoplay=True)  # 替换为实际采样率
    else:
        print(f"Audio file for ID {id} not found.")

# 调用方式
id_to_play =6832 # 替换成你想要播放的ID
play_wav_by_id(id_to_play, PATH_CONFIG['save_dir'], data)


计算时长

In [ ]:
import os
import librosa

def calculate_total_duration(folder_path):
    total_duration = 0.0
    wav_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.wav')]
    
    for filepath in wav_files:
        try:
            y, sr = librosa.load(filepath, sr=None)
            total_duration += librosa.get_duration(y=y, sr=sr)
        except Exception as e:
            print(f"处理文件 {filepath} 时出错: {e}")

    return total_duration / 3600.0  # 转换为小时

if __name__ == "__main__":
    folder_path = '/data/jinming/LLM/word/word_rep'  # 替换为你的文件夹路径
    total_duration = calculate_total_duration(folder_path)
    print(f"总时长: {total_duration:.2f} 小时")


## LJ Speech

hps

In [ ]:

# 超参数配置
hps = utils.get_hparams_from_file("./configs/ljs_base.json")
# 初始化模型
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda(0)
_ = net_g.eval()
# 加载预训练模型
_ = utils.load_checkpoint("./pretrained/pretrained_ljs.pth", net_g, None)

In [ ]:
import scipy.io.wavfile
def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)  # text_to_sequence_phn
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

stn_tst = get_text('''I'm sorry, but your flight has delayed due to , to , severe weather conditions.''', hps) #ðə kˌɑːmpjuːtˈeɪʃənəl ɐnˈæləsˌɪs ɹᵻkwˈaɪɚz ˌɑːptɪmᵻzˈeɪʃən
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1.0)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
print(x_tst)
print(x_tst_lengths)


# 保存音频为 WAV 文件
#scipy.io.wavfile.write('output.wav', hps.data.sampling_rate, audio)

In [ ]:
import torch
import numpy as np
import IPython.display as ipd
import scipy.io.wavfile

def get_text(text, hps):
    text_norm = text_to_sequence_phn(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

# 示例文本
text = '''ðɪ ɛkspˈiəɹɪənst ʃˈɛf pɹɪpˈɛɹd ɐn ɛkskwˈɪsɪ mˈiːl'''
stn_tst = get_text(text, hps)

with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()

    # 获取音素时长预测
    x, m_p, logs_p, x_mask = net_g.enc_p(x_tst, x_tst_lengths)
    if net_g.n_speakers > 0:
        sid = torch.LongTensor([0]).cuda()  # 假设说话人 ID 为 0
        g = net_g.emb_g(sid).unsqueeze(-1)
    else:
        g = None

    if net_g.use_sdp:
        logw = net_g.dp(x, x_mask, g=g, reverse=True, noise_scale=0.8)
    else:
        logw = net_g.dp(x, x_mask, g=g)
    w = torch.exp(logw) * x_mask

    # 自定义调整音素时长
    # 创建与 w 相同形状的倍率张量
    duration_adjust = torch.ones_like(w)
    # 示例：将第 3 个音素的时长加倍
    duration_adjust[:, :, 2] = 2.0
    # 应用自定义时长
    custom_lengths = w * duration_adjust

    # 调用 infer 方法，传入 custom_lengths
    audio = net_g.infer(
        x_tst, x_tst_lengths, sid=sid, noise_scale=0.667, noise_scale_w=0.8,
        length_scale=1.0, custom_lengths=custom_lengths
    )[0][0, 0].data.cpu().float().numpy()

# 播放音频
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

# 保存音频为 WAV 文件
#scipy.io.wavfile.write('output_custom_duration.wav', hps.data.sampling_rate, audio)

## VCTK

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
import random
import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence, text_to_sequence_phn

from scipy.io.wavfile import write


hps = utils.get_hparams_from_file("./configs/vctk_base.json")
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda(0)
_ = net_g.eval()

_ = utils.load_checkpoint("./pretrained/pretrained_vctk.pth", net_g, None)


def get_text(text, hps):
    text_norm = text_to_sequence_phn(text, hps.data.text_cleaners)  # text_to_sequence_phn
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

stn_tst = get_text('''kˈɔ...kˈɔfi kˈiːps mˌiː ɐwˈeɪk''', hps) #ðə blˈuː skˈaɪ ʃˈoʊz θɹuː ðə wˈɪndoʊ
with torch.no_grad():                                #ðə b'lu: skaɪ ʃoʊz θru: ðə 'windoʊ
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([0]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=0.6, noise_scale_w=0.6, length_scale=1.1)[0][0,0].data.cpu().float().numpy()

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))


DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


/home/jinming/miniconda3/envs/vits/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


INFO:root:Loaded checkpoint './pretrained/pretrained_vctk.pth' (iteration 0)


### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))

In [ ]:
import matplotlib.pyplot as plt
import IPython.display as ipd
import csv
import os
import json
import math
import torch
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
from scipy.io.wavfile import write
import threading
import gc

class ModelWrapper:
    def __init__(self, model_path, config_path, device, model_id):
        self.device = device
        self.model_id = model_id
        self.hps = utils.get_hparams_from_file(config_path)
        self.model = self._load_model(model_path)

    def _load_model(self, model_path):
        model = SynthesizerTrn(
            len(symbols),
            self.hps.data.filter_length // 2 + 1,
            self.hps.train.segment_size // self.hps.data.hop_length,
            **self.hps.model).to(self.device)
        model.eval()
        _ = utils.load_checkpoint(model_path, model, None)
        return model

    def get_text(self, text):
        text_norm = text_to_sequence(text, self.hps.data.text_cleaners)
        if self.hps.data.add_blank:
            text_norm = commons.intersperse(text_norm, 0)
        text_norm = torch.LongTensor(text_norm)
        return text_norm

    def get_word_alignments(self, attn, text, audio, hop_length, sampling_rate):
        if not text.strip():
            return []
                
        words = text.split()
        if not words:
            return []
        
        # 打印调试信息
        audio_samples = len(audio)
        audio_duration = audio_samples / sampling_rate
        print(f"Audio length: {audio_duration:.3f}s ({audio_samples} samples)")
        print(f"Sample rate: {sampling_rate}Hz")
        print(f"Hop length: {hop_length}")
        print(f"Total frames in attention: {attn.shape[0]}")
        
        # 计算帧级别的注意力权重
        attention_weights = np.sum(attn, axis=1)
        attention_weights = attention_weights / np.sum(attention_weights)
        cumsum = np.cumsum(attention_weights)
        
        word_timestamps = []
        for i, word in enumerate(words):
            # 计算归一化位置
            start_pos = i / len(words)
            end_pos = (i + 1) / len(words)
            
            # 找到对应帧位置
            start_frame = np.searchsorted(cumsum, start_pos)
            end_frame = np.searchsorted(cumsum, end_pos)
            
            # 转换为时间
            start_time = start_frame * hop_length / sampling_rate
            end_time = end_frame * hop_length / sampling_rate
            
            word_timestamps.append({
                'word': word,
                'start': round(float(start_time), 3),
                'end': round(float(end_time), 3),
                'duration': round(float(end_time - start_time), 3)
            })
        
        return word_timestamps
    
    # 相应地修改 synthesize 方法:
    def synthesize(self, text):
        with torch.no_grad():
            x_tst = self.get_text(text).unsqueeze(0).to(self.device)
            x_tst_lengths = torch.LongTensor([x_tst.size(1)]).to(self.device)
            
            # 调整length_scale以生成更多帧
            audio, attn, *_ = self.model.infer(
                x_tst, 
                x_tst_lengths,
                noise_scale=.667,
                noise_scale_w=0.6,
                length_scale=1.2  # 增加length_scale值
            )
            
            audio = audio[0][0].cpu().float().numpy()
            attn = attn[0].cpu().numpy()
            
            # 确保生成了足够的attention frames
            print(f"Generated attention matrix shape: {attn.shape}")
            
            alignments = self.get_word_alignments(
                attn,
                text,
                audio,
                self.hps.data.hop_length,
                self.hps.data.sampling_rate
            )
            
            return audio, alignments, attn

class AudioGenerator:
    def __init__(self, models, save_dir):
        self.models = models
        self.save_dir = save_dir
        self.audio_dir = os.path.join(save_dir, 'wavs')
        self.label_dir = os.path.join(save_dir, 'labels')
        self.attn_dir = os.path.join(save_dir, 'attention')
        
        os.makedirs(self.audio_dir, exist_ok=True)
        os.makedirs(self.label_dir, exist_ok=True)
        os.makedirs(self.attn_dir, exist_ok=True)

    def save_attention_plot(self, attn, text, save_path):
        plt.figure(figsize=(12, 8))
        plt.imshow(attn.T, aspect='auto', origin='lower')
        plt.title(f'Attention Matrix: {text[:50]}...')
        plt.colorbar()
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

    def process_entry(self, entry, model_wrapper):
        try:
            id = entry['id']
            text = entry.get('dysfluent', '').strip()
            
            if not text:
                print(f"Skipping ID {id}: Empty text")
                return
                
            print(f"Processing ID {id} with Model {model_wrapper.model_id}: {text}")
            
            audio, alignments, attn = model_wrapper.synthesize(text)
            
            # 保存音频
            audio_path = os.path.join(self.audio_dir, f"{id}.wav")
            write(audio_path, model_wrapper.hps.data.sampling_rate, audio)
            
            # 保存标签
            label_path = os.path.join(self.label_dir, f"{id}.json")
            with open(label_path, 'w', encoding='utf-8') as f:
                json.dump({
                    "id": id,
                    "dysfluent": text,
                    "correct": entry.get('correct', ''),
                    "label": entry.get('label', ''),
                    "category": entry.get('category', ''),
                    "words": alignments
                }, f, ensure_ascii=False, indent=2)
            
            # 保存注意力矩阵图
            attn_path = os.path.join(self.attn_dir, f"{id}_attention.png")
            self.save_attention_plot(attn, text, attn_path)
            
            print(f"Saved audio: {audio_path}")
            print(f"Saved label: {label_path}")
            print(f"Saved attention plot: {attn_path}")
            
            del audio, alignments, attn
            torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"Error processing ID {id}: {str(e)}")
            import traceback
            print(traceback.format_exc())

    def generate_wavs_in_range(self, start_id, end_id, data):
        entries_in_range = [entry for entry in data if start_id <= entry['id'] <= end_id]
        for entry in entries_in_range:
            self.process_entry(entry, self.models[0])

def main():
    json_file_path = '/data/jinming/LLM/word/word_del_1.json'
    save_dir = '/data/jinming/LLM/test'
    
    device = 'cuda:0'
    model_configs = [
        {
            'model_path': './pretrained/pretrained_ljs.pth',
            'config_path': './configs/ljs_base.json',
            'device': device,
            'model_id': 0
        }
    ]

    models = [ModelWrapper(**config) for config in model_configs]

    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    generator = AudioGenerator(models, save_dir)
    start_id = 1
    end_id = 10
    
    generator.generate_wavs_in_range(start_id, end_id, data)

if __name__ == "__main__":
    main()